<a href="https://colab.research.google.com/github/miiiingi/dacon_sentiment/blob/main/dacon_sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')
my_folder = '/gdrive/MyDrive/ColabNotebooks/dacon_senti/dataset/dataset'

Mounted at /gdrive/


In [ ]:
import pandas as pd
import numpy as np
!pip install konlpy
from konlpy.tag import Mecab
from konlpy.tag import Okt
from tqdm import tqdm
from torchtext.legacy import data, datasets
import torch
import random
import os
import math
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
SEED = 722
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_all(SEED)

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 71.2 MB/s 


Torchtext - Field 정의

In [ ]:
tokenizer = Okt()
ID = data.Field(sequential=False, use_vocab=False)
TEXT = data.Field(sequential=True, use_vocab=True, tokenize=tokenizer.morphs)
LABEL = data.LabelField(sequential=False, use_vocab=False, is_target=True)

첫 번째 실험 : tokenizer을 morphs(75.37 / clip grad norm x > 과적합 심하게 발생) > nouns(68.4)로 변화 / morphs \\
두 번째 실험 : clip grad norm(73.9 > 과적합 발생)


Torchtext - 데이터셋 불러와서 정의

In [ ]:
train_data = data.TabularDataset(
                            path = f'{my_folder}/train.csv',
                            format = 'csv',
                            fields = [('id', ID), ('document', TEXT), ('label', LABEL)],
                            skip_header = True,)
test_data = data.TabularDataset(
                            path = f'{my_folder}/test.csv',
                            format = 'csv',
                            fields = [('id', ID), ('document', TEXT)],
                            skip_header = True,)
train_data, valid_data = train_data.split(split_ratio = 0.5)

Torchtext - 단어 집합 만들기

In [ ]:
TEXT.build_vocab(train_data, min_freq=5, max_size= 1000) # 단어 집합 생성
# LABEL.build_vocab(train_data)

In [ ]:
vars(train_data[1])

{'document': ['우리나라', '좀비', '영화', '는', '왜', '이러냐', '..'],
 'id': '1494',
 'label': '0'}

Torchtext - 배치화 시키기

In [ ]:
batch_size = 64
train_loader = data.Iterator(train_data, batch_size=batch_size, shuffle=True)
val_loader = data.Iterator(valid_data, batch_size=batch_size, train=False, sort=False)
test_loader = data.Iterator(test_data, batch_size=batch_size, train=False, sort=False)

In [ ]:
print(next(iter(train_loader)))
print(next(iter(val_loader)))
print(next(iter(test_loader)))


[torchtext.legacy.data.batch.Batch of size 64]
	[.id]:[torch.LongTensor of size 64]
	[.document]:[torch.LongTensor of size 21x64]
	[.label]:[torch.LongTensor of size 64]

[torchtext.legacy.data.batch.Batch of size 64]
	[.id]:[torch.LongTensor of size 64]
	[.document]:[torch.LongTensor of size 21x64]
	[.label]:[torch.LongTensor of size 64]

[torchtext.legacy.data.batch.Batch of size 64]
	[.id]:[torch.LongTensor of size 64]
	[.document]:[torch.LongTensor of size 19x64]


Embedding layer 어떻게 처리할 지 생각해보자

Model 정의

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        even_div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        odd_div_term = torch.exp(torch.arange(1, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * even_div_term)
        pe[:, 1::2] = torch.cos(position * odd_div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        pe.requires_grad = False
        self.register_buffer('pe', pe)
    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class Transformer(nn.Module):
    def __init__(self, n_vocab, n_embed_pre, n_embed_post, n_layers, n_head, dropout):
        super(Transformer, self).__init__()
        self.src_mask = None
        self.embed = nn.Embedding(n_vocab, n_embed_post)
        self.pos_enc = PositionalEncoding(n_embed_post)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=n_embed_post, nhead = n_head, dropout=dropout)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=n_layers)        
        self.n_embed_post = n_embed_post
        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.embed.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            self.src_mask = self._generate_square_subsequent_mask(src.shape[0]).to(src.device)
        src = self.embed(src)
        src = self.pos_enc(src)
        output = self.encoder(src, self.src_mask)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [ ]:
from torch import nn as nn 
import torch.nn.functional as F
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, model, dropout=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        # self.embed = nn.Embedding(n_vocab, embed_dim)
        self.embed = model
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers * 2,
                          bidirectional = True)
        self.out = nn.Linear(self.hidden_dim * 2, n_classes)

    def forward(self, x):
        x = self.embed(x)
        # h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x, _ = self.gru(x)  # GRU의 리턴값은 (시퀀스 길이, 배치 크기, 은닉 상태의 크기)
        h_t = x[-1, :, :] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transformer = Transformer(n_vocab = len(TEXT.vocab),n_embed_pre = 25, n_embed_post = 128, n_layers = 1, n_head = 8, dropout = 0).to(DEVICE)
model = GRU(n_layers = 1, hidden_dim = 256, n_vocab = len(TEXT.vocab), embed_dim = 128, n_classes = 2, model = transformer, dropout = 0.3).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.document.to(DEVICE), batch.label.long().to(DEVICE)
        # print(x.shape)
        # print(model(x).shape)
        # print(y.shape)
        logit = model(x)
        # print(logit.shape)
        # print(y.shape)
        loss = F.cross_entropy(logit, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    with torch.no_grad():
      for batch in val_iter:
          x, y = batch.document.to(DEVICE), batch.label.long().to(DEVICE)
          logit = model(x)
          loss = F.cross_entropy(logit, y)
          total_loss += loss.item()
          corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
      size = len(val_iter.dataset)
      avg_loss = total_loss / size
      avg_accuracy = 100.0 * corrects / size
      return avg_loss, avg_accuracy

In [ ]:
def evaluate_test(model, test_loader, device):
    result = []
    model.to(device)
    model.eval()
    with torch.no_grad():
        for batch in tqdm(test_loader):
            x = batch.document.to(device)
            out = model(x)
            pred = torch.argmax(out, dim=1)
            result.append(pred)
    return torch.cat(result).cpu().numpy()

In [ ]:
best_val_loss = None
val_accuracy_accum = 0
Epochs = 100
for e in tqdm(range(Epochs)):
    train(model, optimizer, train_loader)
    train_loss, train_accuracy = evaluate(model, train_loader)
    val_loss, val_accuracy = evaluate(model, val_loader)
    val_accuracy_accum += val_accuracy

    print("[Epoch: %d] train loss : %5.2f | train acc : %5.2f | val loss : %5.2f | val accuracy : %5.2f" % (e+1, train_loss, train_accuracy, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("/gdrive/MyDrive/ColabNotebooks/dacon_senti/snapshot"):
            os.makedirs("/gdrive/MyDrive/ColabNotebooks/dacon_senti/snapshot")
        torch.save(model.state_dict(), f'/gdrive/MyDrive/ColabNotebooks/dacon_senti/snapshot/sentiment.pt')
        best_val_loss = val_loss
print(f'final validation score : {val_accuracy_accum / Epochs}')

  1%|          | 1/100 [00:02<03:24,  2.06s/it]

[Epoch: 1] train loss :  0.01 | train acc : 51.20 | val loss :  0.01 | val accuracy : 51.36


  2%|▏         | 2/100 [00:03<03:02,  1.87s/it]

[Epoch: 2] train loss :  0.01 | train acc : 48.80 | val loss :  0.01 | val accuracy : 48.64


  3%|▎         | 3/100 [00:05<02:55,  1.81s/it]

...

[Epoch: 98] train loss :  0.00 | train acc : 99.76 | val loss :  0.02 | val accuracy : 78.24


 99%|█████████▉| 99/100 [02:44<00:01,  1.64s/it]

[Epoch: 99] train loss :  0.00 | train acc : 99.68 | val loss :  0.02 | val accuracy : 78.52


100%|██████████| 100/100 [02:46<00:00,  1.67s/it]

[Epoch: 100] train loss :  0.00 | train acc : 98.76 | val loss :  0.02 | val accuracy : 78.24
final validation score : 73.77840423583984


검증 데이터셋으로 구한 초모수를 통해 전체 데이터셋을 훈련시켜 테스트에 사용

In [ ]:
train_data = data.TabularDataset(
                            path = f'{my_folder}/train.csv',
                            format = 'csv',
                            fields = [('id', ID), ('document', TEXT), ('label', LABEL)],
                            skip_header = True,)
test_data = data.TabularDataset(
                            path = f'{my_folder}/test.csv',
                            format = 'csv',
                            fields = [('id', ID), ('document', TEXT)],
                            skip_header = True,)

In [ ]:
TEXT.build_vocab(train_data, min_freq=5, max_size= 1000) # 단어 집합 생성
# LABEL.build_vocab(train_data)

In [ ]:
batch_size = 64
train_loader = data.Iterator(train_data, batch_size=batch_size, shuffle=True)
test_loader = data.Iterator(test_data, batch_size=batch_size, train=False, sort=False)

In [ ]:
print(next(iter(train_loader)))
print(next(iter(test_loader)))


[torchtext.legacy.data.batch.Batch of size 64]
	[.id]:[torch.LongTensor of size 64]
	[.document]:[torch.LongTensor of size 20x64]
	[.label]:[torch.LongTensor of size 64]

[torchtext.legacy.data.batch.Batch of size 64]
	[.id]:[torch.LongTensor of size 64]
	[.document]:[torch.LongTensor of size 19x64]


In [ ]:
transformer = Transformer(n_vocab = len(TEXT.vocab),n_embed_pre = 25, n_embed_post = 128, n_layers = 1, n_head = 8, dropout = 0).to(DEVICE)
model = GRU(n_layers = 1, hidden_dim = 256, n_vocab = len(TEXT.vocab), embed_dim = 128, n_classes = 2, model = transformer, dropout = 0.3).to(DEVICE)
model.load_state_dict(torch.load(f'/gdrive/MyDrive/ColabNotebooks/dacon_senti/snapshot/sentiment.pt'))
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
Epochs = 100
for e in tqdm(range(Epochs)):
    train(model, optimizer, train_loader)
    train_loss, train_accuracy = evaluate(model, train_loader)
torch.save(model.state_dict(), f'/gdrive/MyDrive/ColabNotebooks/dacon_senti/snapshot/sentiment.pt')

RuntimeError: ignored

TEST 진행

In [ ]:
model.load_state_dict(torch.load(f'/gdrive/MyDrive/ColabNotebooks/dacon_senti/snapshot/sentiment.pt'))
result = evaluate_test(model, test_loader, DEVICE)

100%|██████████| 79/79 [00:00<00:00, 92.82it/s] 


In [ ]:
submission = pd.read_csv(f"{my_folder}/sample_submission.csv")
submission["label"] = result
print(submission)
submission.to_csv(f"{my_folder}/submission.csv", index=False)

        id  label
0        1      0
1        2      1
2        3      1
3        4      1
4        5      1
...    ...    ...
4995  4996      0
4996  4997      0
4997  4998      1
4998  4999      0
4999  5000      0

[5000 rows x 2 columns]
